In [6]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests
from bs4 import BeautifulSoup
import re
import folium

In [8]:
url = 'https://www.wikitree.co.kr/main/news_view.php?id=217101'
response = requests.get(url)
#response.text

In [9]:
resp = response.text.replace('&nbsp;',' ')
soup = BeautifulSoup(resp)
#soup

In [10]:
raw_data = soup.select_one('div#wikicon')
#raw_data

In [11]:
title_one = raw_data.select_one('div strong')
title_one

<strong>1. 44회 다성일식</strong>

In [12]:
title_all = raw_data.select('div strong')
first_title_list = []
for title_one in title_all:
    title_text = title_one.text
    if title_text:
        if title_text.strip() == '14.' or title_text.strip() == '20.':
            continue
        else:
            first_title_list.append(title_text)
        

In [13]:
#first_title_list
second_title_list = []
for title_one in first_title_list:
    second_title_list.append(title_one.split('회 ')[1].strip())

In [14]:
title_list = second_title_list

In [15]:
# 전화번호 찾기 1 실패
tel_p = re.compile('\d{2,3}-\d{3,4}-\d{4}')
tel_all = raw_data.select('div ~ p')
tel_list = []
for item in tel_all:
    str_item = str(item)
    info = tel_p.findall(str_item)
    if info:
        tel_list.append(info)
        
len(tel_list)

19

In [16]:
# 전화번호 찾기 2 성공
tel_p = re.compile('\d{2,3}-\d{3,4}-\d{4}')
tel_all = raw_data.select('p')
tel_list = []
for item in tel_all:
    str_item = str(item)
    info = tel_p.findall(str_item)
    if info:
        tel_list.append(info)
        
len(tel_list)

20

In [17]:
# 전화번호 찾기 3 성공
tel_p = re.compile('\d{2,3}-\d{3,4}-\d{4}')
tel_data = raw_data.find_all(string = tel_p)
tel_list = []
for tel_one in tel_data:
    tel_list.append(tel_one.strip())
    
print(len(tel_list))

20


In [18]:
# 주소 찾기 
address_p = re.compile('^.*구\s.*동')
address_list = raw_data.find_all(string = address_p)
print(address_list)

['서울시 서대문구 창천동 72-36', '서울시 용산구 용산동3가 1-21', '서울시 영등포구 여의도동 44-37 아일렉스타워15층(스카이점)', '서울시 광진구 화양동 9-22', '서울시 강북구 수유동 563-14', '서울시 성북구 동소문동 1가 62', '서울시 강남구 역삼동 821 이즈타워 B3', '서울시 마포구 아현동  282-1', '서울시 강남구 논현동 98-12', '서울시 강남구 청담동 80-21', '서울시 강남구 신사동 526-11', '서울시 서초구 반포4동 82-22', '서울시 동대문구 제기동 138-2', '서울시 종로구 누하동 25-1', '서울시 서초구 양재동 1-28', '서울시 종로구 부암동 208-37', '종로구 당주동 5 로얄빌딩 지하 1층', '서울 마포구 연남동 257-8 ', '서울 중구 필동 1가 3-20', '서울 중구 북창동 93-36']


In [19]:
restaurant_list = []
for i in range(len(title_list)):
    info = [title_list[i], tel_list[i], address_list[i]]
    restaurant_list.append(info)
    


In [20]:
import pickle
import sys
sys.setrecursionlimit(10000)

In [21]:
with open('restaurant_list.pkl', 'wb') as f:
    pickle.dump(restaurant_list, f)

In [22]:
with open("restaurant_list.pkl","rb") as fr:
    data = pickle.load(fr)
#print(data)


In [23]:
# Kakao API : 구주소 -> 신주소, 위경도 
def getLatLng(address):
    #address = '서울특별시 구로구 디지털로 306'
    app_key = 'KakaoAK ' + '0f4891619d301962183c7d4d00948072'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query':address,
        'page':'',
        'AddressSize':''
    }
    headers = {'Authorization':app_key}
    response = requests.get(url, headers=headers, params=params)
    status_code = response.status_code
    if status_code == 200:
        address_info = response.json()
        x = address_info['documents'][0]['x']
        y = address_info['documents'][0]['y']
        return (y, x)
    else:
        return (0, 0)

In [24]:
# 종로구 당주동 5 로얄빌딩 지하 1층 위경도가 않옴
print(data[16][2])
data[16][2] = data[16][2].split('로얄빌딩')[0].strip()
print(data[16][2])

종로구 당주동 5 로얄빌딩 지하 1층
종로구 당주동 5


In [25]:
location = list() # 위, 경도 저장
for item in data:
    #print(item)
    ret_location = getLatLng(item[2])
    #print(ret_location)
    #print(location)
    location.append(ret_location)
    
location

[('37.5568947886341', '126.932965561024'),
 ('37.5344088960778', '126.974989040272'),
 ('37.5189272446286', '126.930731521429'),
 ('37.5414828193382', '127.069272933219'),
 ('37.6456327086471', '127.007127542151'),
 ('37.5900669779109', '127.00839172712'),
 ('37.4986090348489', '127.027897674019'),
 ('37.5551587300272', '126.955664169252'),
 ('37.5204072521637', '127.036095252922'),
 ('37.5261747199815', '127.044857053316'),
 ('37.5210732573275', '127.019786406742'),
 ('37.4947067319368', '127.00090838481'),
 ('37.5862034507119', '127.034542600462'),
 ('37.5801412950445', '126.969021243655'),
 ('37.4844942544155', '127.039180053795'),
 ('37.5955770447834', '126.964055305288'),
 ('37.5719883882772', '126.974327965082'),
 ('37.5620838593197', '126.923556958229'),
 ('37.5617589423156', '126.992201358274'),
 ('37.562416037119', '126.978922667995')]

In [26]:
len(location)

20

In [27]:
for i in range(len(data)):
    data[i].append(location[i])
    

In [28]:
data

[['다성일식',
  '02-338-8951',
  '서울시 서대문구 창천동 72-36',
  ('37.5568947886341', '126.932965561024')],
 ['봉산집',
  '02-793-5022',
  '서울시 용산구 용산동3가 1-21',
  ('37.5344088960778', '126.974989040272')],
 ['창고43',
  '02-786-5959',
  '서울시 영등포구 여의도동 44-37 아일렉스타워15층(스카이점)',
  ('37.5189272446286', '126.930731521429')],
 ['돕감자탕',
  '02-499-2838',
  '서울시 광진구 화양동 9-22',
  ('37.5414828193382', '127.069272933219')],
 ['대보명가',
  '02-907-6998',
  '서울시 강북구 수유동 563-14',
  ('37.6456327086471', '127.007127542151')],
 ['해뜨는집',
  '02-764-6354',
  '서울시 성북구 동소문동 1가 62',
  ('37.5900669779109', '127.00839172712')],
 ['아이 해브어 드림',
  '02-3453-7696',
  '서울시 강남구 역삼동 821 이즈타워 B3',
  ('37.4986090348489', '127.027897674019')],
 ['아현동 간장게장',
  '02-312-7530',
  '서울시 마포구 아현동  282-1',
  ('37.5551587300272', '126.955664169252')],
 ['왕소금구이',
  '02-545-6844',
  '서울시 강남구 논현동 98-12',
  ('37.5204072521637', '127.036095252922')],
 ['라틀리에 모니크',
  '02-549-9210',
  '서울시 강남구 청담동 80-21',
  ('37.5261747199815', '127.044857053316')],
 ['비스트로 딩

In [29]:
data[0][3]

('37.5568947886341', '126.932965561024')

In [30]:
# 지도 생성
map = folium.Map(location=data[0][3], zoom_start=11)
map

In [31]:
data[0]

['다성일식',
 '02-338-8951',
 '서울시 서대문구 창천동 72-36',
 ('37.5568947886341', '126.932965561024')]

In [32]:
# 마커 생성
folium.Marker(location=data[0][3], 
              popup=data[0][1],
             icon=folium.Icon(color='pink', icon='glyphicon glyphicon-road')).add_to(map)
map

In [33]:
for i in range(len(data)):
    folium.Marker(location=data[i][3], 
              popup='Tel: ' + data[i][1],
             icon=folium.Icon(color='red', icon='glyphicon glyphicon-time')).add_to(map)
map
